<a href="https://colab.research.google.com/github/DevzsJhonny/Colab_notebooks/blob/main/PySpark_Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install pyspark

In [25]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").getOrCreate()

# Spark SQL - Consultas e Seleções


In [26]:
df = spark.sql('''select 'OK' as Status''')
df.show()

+------+
|Status|
+------+
|    OK|
+------+



# Importando Dados

In [27]:
df = spark.read.csv('cereal.csv', sep = ',', inferSchema=True, header = True)
df.show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [28]:
df.count()

77

# Manipulando Dados com Spark SQL

In [29]:
df.createOrReplaceTempView("cereal") # cria uma tabela temporia dentro da Sessão, afim de manipular os dados

In [30]:
cereal = spark.sql('''SELECT COUNT(*) AS total FROM cereal WHERE mfr = 'G' ''')
cereal.show()

+-----+
|total|
+-----+
|   22|
+-----+



In [31]:
cereal = spark.sql('''SELECT * FROM cereal WHERE mfr = 'G' ''')
cereal.show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|Apple Cinnamon Ch...|  G|   C|     110|      2|  2|   180|  1.5| 10.5|    10|    70|      25|    1|   1.0|0.75|29.509541|
|             Basic 4|  G|   C|     130|      3|  2|   210|  2.0| 18.0|     8|   100|      25|    3|  1.33|0.75|37.038562|
|            Cheerios|  G|   C|     110|      6|  2|   290|  2.0| 17.0|     1|   105|      25|    1|   1.0|1.25|50.764999|
|Cinnamon Toast Cr...|  G|   C|     120|      1|  3|   210|  0.0| 13.0|     9|    45|      25|    2|   1.0|0.75|19.823573|
|            Clusters|  G|   C|     110|      3|  2|   140|  2.0| 13.0|     7|   105|      25|    3|   1.0| 0.5|40.400208|
|         Cocoa 

In [32]:
df.where(df['mfr']=='G').count()

22

In [33]:
df.where(df['type']=='C').count()

74

# Select Distinct no SparkSQL

In [34]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



In [35]:
df.createOrReplaceTempView("cereal")

In [36]:
cereal = spark.sql('''SELECT  DISTINCT  type, mfr FROM cereal''')
cereal.show()

+----+---+
|type|mfr|
+----+---+
|   C|  P|
|   C|  Q|
|   C|  N|
|   H|  Q|
|   C|  R|
|   H|  N|
|   C|  G|
|   H|  A|
|   C|  K|
+----+---+



# Where no SparkSQL

In [37]:
cereal = spark.sql('''SELECT  DISTINCT  type, mfr FROM cereal where mfr in( 'N', 'K' ) ''')
cereal.show()

+----+---+
|type|mfr|
+----+---+
|   C|  N|
|   H|  N|
|   C|  K|
+----+---+



In [38]:
cereal = spark.sql('''SELECT  * FROM cereal where mfr = 'K' AND calories = 100 ''')
cereal.show()

+-------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|               name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+-------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|        Corn Flakes|  K|   C|     100|      2|  0|   290|  1.0| 21.0|     2|    35|      25|    1|   1.0| 1.0|45.863324|
|Frosted Mini-Wheats|  K|   C|     100|      3|  0|     0|  3.0| 14.0|     7|   100|      25|    2|   1.0| 0.8|58.345141|
|         Product 19|  K|   C|     100|      3|  0|   320|  1.0| 20.0|     3|    45|     100|    3|   1.0| 1.0| 41.50354|
+-------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+



In [39]:
cereal = spark.sql('''SELECT  * FROM cereal where  calories = 100 ''')
cereal.show()
cereal.count()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|         Corn Flakes|  K|   C|     100|      2|  0|   290|  1.0| 21.0|     2|    35|      25|    1|   1.0| 1.0|45.863324|
|Cream of Wheat (Q...|  N|   H|     100|      3|  0|    80|  1.0| 21.0|     0|    -1|       0|    2|   1.0| 1.0|64.533816|
|Crispy Wheat & Ra...|  G|   C|     100|      2|  1|   140|  2.0| 11.0|    10|   120|      25|    3|   1.0|0.75|36.176196|
|         Double Chex|  R|   C|     100|      2|  0|   190|  1.0| 18.0|     5|    80|      25|    3|   1.0|0.75|44.330856|
| Frosted Mini-Wheats|  K|   C|     100|      3|  0|     0|  3.0| 14.0|     7|   100|      25|    2|   1.0| 0.8|58.345141|
|        Golden 

17

# GROUP BY

In [40]:
cereal = spark.sql('''SELECT mfr, \
                             type, \
                             count(*) AS total,
                             sum(calories) AS total_calories
                      FROM cereal
                      GROUP BY  \
                            mfr, \
                            type ''''')
cereal.show()

+---+----+-----+--------------+
|mfr|type|total|total_calories|
+---+----+-----+--------------+
|  A|   H|    1|           100|
|  P|   C|    9|           980|
|  K|   C|   23|          2500|
|  G|   C|   22|          2450|
|  Q|   C|    7|           660|
|  R|   C|    8|           920|
|  Q|   H|    1|           100|
|  N|   H|    1|           100|
|  N|   C|    5|           420|
+---+----+-----+--------------+



# CASE WHEN

In [41]:
cereal = spark.sql(''' SELECT DISTINCT type
                        FROM cereal''')
cereal.show()

+----+
|type|
+----+
|   C|
|   H|
+----+



In [42]:
cereal = spark.sql(''' SELECT mfr,
                              type,
                             (case
                                    when type = 'C' then 'A'
                                    --when type = 'H' then 'B'
                                    else 'B'
                             end) as type_new,
                             count(*) AS total,
                             sum(calories) AS total_calories
                      FROM cereal
                      GROUP BY
                            mfr,
                            type ''''')
cereal.show()

+---+----+--------+-----+--------------+
|mfr|type|type_new|total|total_calories|
+---+----+--------+-----+--------------+
|  A|   H|       B|    1|           100|
|  P|   C|       A|    9|           980|
|  K|   C|       A|   23|          2500|
|  G|   C|       A|   22|          2450|
|  Q|   C|       A|    7|           660|
|  R|   C|       A|    8|           920|
|  Q|   H|       B|    1|           100|
|  N|   H|       B|    1|           100|
|  N|   C|       A|    5|           420|
+---+----+--------+-----+--------------+



# Consultas Avançadas em SQL usando PySpark


In [43]:
df.show(5)

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
+---------------

In [44]:
cereal = spark.sql('''
                  SELECT mfr,
                         type,
                         sum(calories) as sum_calories,
                         min(calories) as min_calories,
                         max(calories) as max_calories,
                         cast (avg(calories) as decimal(10,2)) as avg_calories,

                         sum(carbo) as sum_carbo,
                         min(carbo) as min_carbo,
                         max(carbo) as max_carbo,
                         cast (avg(carbo) as decimal(10,2)) as avg_carbo,

                         sum(vitamins) as sum_vitamins,
                         min(vitamins) as min_vitamins,
                         max(vitamins) as max_vitamins,
                         cast (avg(vitamins) as decimal(10,2)) as avg_vitamins,

                         count(distinct name) as count_distinct_names,
                         count(name) as count_names
                  FROM cereal
                  GROUP BY mfr, type
                  ORDER BY mfr, type ''')
cereal.show()

+---+----+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|mfr|type|sum_calories|min_calories|max_calories|avg_calories|sum_carbo|min_carbo|max_carbo|avg_carbo|sum_vitamins|min_vitamins|max_vitamins|avg_vitamins|count_distinct_names|count_names|
+---+----+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|  A|   H|         100|         100|         100|      100.00|     16.0|     16.0|     16.0|    16.00|          25|          25|          25|       25.00|                   1|          1|
|  G|   C|        2450|         100|         140|      111.36|    324.0|     10.5|     21.0|    14.73|         775|          25|         100|       35.23|                  22|         22|
|  K|   C|        2500|          50|         160|      108.7

In [48]:
cereal = spark.sql('''
                  SELECT mfr,
                         type,
                         (case
                              when mfr = 'A' then 'Abacaxi'
                              when mfr = 'G' then 'Goiaba'
                              when mfr = 'K' then 'Banana'
                              when mfr = 'N' then 'Maça'
                              when mfr = 'P' then 'Tomate'
                              when mfr = 'Q' then 'Pera'
                              when mfr = 'R'then 'Uva'
                              else 'NA'
                         end) as type_fruit,

                         sum(calories) as sum_calories,
                         min(calories) as min_calories,
                         max(calories) as max_calories,
                         cast (avg(calories) as decimal(10,2)) as avg_calories,

                         sum(carbo) as sum_carbo,
                         min(carbo) as min_carbo,
                         max(carbo) as max_carbo,
                         cast (avg(carbo) as decimal(10,2)) as avg_carbo,

                         sum(vitamins) as sum_vitamins,
                         min(vitamins) as min_vitamins,
                         max(vitamins) as max_vitamins,
                         cast (avg(vitamins) as decimal(10,2)) as avg_vitamins,

                         count(distinct name) as count_distinct_names,
                         count(name) as count_names
                  FROM cereal
                  GROUP BY mfr, type
                  ORDER BY mfr, type ''')
cereal.show()

+---+----+----------+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|mfr|type|type_fruit|sum_calories|min_calories|max_calories|avg_calories|sum_carbo|min_carbo|max_carbo|avg_carbo|sum_vitamins|min_vitamins|max_vitamins|avg_vitamins|count_distinct_names|count_names|
+---+----+----------+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|  A|   H|   Abacaxi|         100|         100|         100|      100.00|     16.0|     16.0|     16.0|    16.00|          25|          25|          25|       25.00|                   1|          1|
|  G|   C|    Goiaba|        2450|         100|         140|      111.36|    324.0|     10.5|     21.0|    14.73|         775|          25|         100|       35.23|                  22|         22|
|  K|

# JOINs

# INNER

In [46]:
sales = spark.read.csv('sales_data_sample.csv', sep = ',', inferSchema=True, header = True)

In [47]:
sales.show(5)

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003 0:00|Shipped| 

In [50]:
sales.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADDRESSLINE1: string (nullable = true)
 |-- ADDRESSLINE2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = tr

In [49]:
sales.createOrReplaceTempView('sales')

In [52]:
calendar = spark.sql('''
                  SELECT DISTINCT orderdate, qtr_id, month_id, year_id
                  FROM sales
                  ORDER BY orderdate
                ''')

sales_data = spark.sql('''
                  SELECT DISTINCT ORDERNUMBER,
                                  CUSTOMERNAME,
                                  ORDERDATE,
                                  SALES,
                                  QUANTITYORDERED,
                                  PRODUCTCODE,
                                  ORDERLINENUMBER,
                                  PRICEEACH
                  FROM sales
                  ORDER BY ORDERNUMBER
                ''')

customers = spark.sql('''
                  SELECT DISTINCT CUSTOMERNAME,
                                  PHONE,
                                  ADDRESSLINE1,
                                  ADDRESSLINE2,
                                  CITY,
                                  STATE,
                                  POSTALCODE,
                                  COUNTRY,
                                  TERRITORY
                  FROM sales
                  ORDER BY CUSTOMERNAME
                ''')
sales_data.createOrReplaceTempView('sales_data')
calendar.createOrReplaceTempView('calendar')
customers.createOrReplaceTempView('customers')

In [53]:
sales_data.show()

+-----------+--------------------+--------------+-------+---------------+-----------+---------------+---------+
|ORDERNUMBER|        CUSTOMERNAME|     ORDERDATE|  SALES|QUANTITYORDERED|PRODUCTCODE|ORDERLINENUMBER|PRICEEACH|
+-----------+--------------------+--------------+-------+---------------+-----------+---------------+---------+
|      10100|Online Diecast Cr...| 1/6/2003 0:00| 3390.0|             50|   S18_2248|              2|     67.8|
|      10100|Online Diecast Cr...| 1/6/2003 0:00| 5151.0|             30|   S18_1749|              3|    100.0|
|      10100|Online Diecast Cr...| 1/6/2003 0:00|1689.03|             49|   S24_3969|              1|    34.47|
|      10100|Online Diecast Cr...| 1/6/2003 0:00|1903.22|             22|   S18_4409|              4|    86.51|
|      10101|Blauer See Auto, Co.| 1/9/2003 0:00| 1404.0|             45|   S24_1937|              3|     31.2|
|      10101|Blauer See Auto, Co.| 1/9/2003 0:00| 3782.0|             25|   S18_2325|              4|   

In [54]:
calendar.show()

+--------------+------+--------+-------+
|     orderdate|qtr_id|month_id|year_id|
+--------------+------+--------+-------+
|1/10/2003 0:00|     1|       1|   2003|
|1/10/2005 0:00|     1|       1|   2005|
|1/12/2004 0:00|     1|       1|   2004|
|1/12/2005 0:00|     1|       1|   2005|
|1/15/2004 0:00|     1|       1|   2004|
|1/16/2004 0:00|     1|       1|   2004|
|1/19/2005 0:00|     1|       1|   2005|
| 1/2/2004 0:00|     1|       1|   2004|
|1/20/2005 0:00|     1|       1|   2005|
|1/22/2004 0:00|     1|       1|   2004|
|1/23/2005 0:00|     1|       1|   2005|
|1/26/2004 0:00|     1|       1|   2004|
|1/26/2005 0:00|     1|       1|   2005|
|1/29/2003 0:00|     1|       1|   2003|
|1/29/2004 0:00|     1|       1|   2004|
|1/31/2003 0:00|     1|       1|   2003|
|1/31/2005 0:00|     1|       1|   2005|
| 1/5/2005 0:00|     1|       1|   2005|
| 1/6/2003 0:00|     1|       1|   2003|
| 1/6/2005 0:00|     1|       1|   2005|
+--------------+------+--------+-------+
only showing top

In [55]:
customers.show()

+--------------------+-----------------+--------------------+------------+--------------+----------+----------+---------+---------+
|        CUSTOMERNAME|            PHONE|        ADDRESSLINE1|ADDRESSLINE2|          CITY|     STATE|POSTALCODE|  COUNTRY|TERRITORY|
+--------------------+-----------------+--------------------+------------+--------------+----------+----------+---------+---------+
|      AV Stores, Co.|   (171) 555-1555|   Fauntleroy Circus|        NULL|    Manchester|      NULL|   EC2 5NT|       UK|     EMEA|
|        Alpha Cognac|       61.77.6555|1 rue Alsace-Lorr...|        NULL|      Toulouse|      NULL|     31000|   France|     EMEA|
|  Amica Models & Co.|      011-4988555| Via Monte Bianco 34|        NULL|        Torino|      NULL|     10100|    Italy|     EMEA|
|Anna's Decoration...|     02 9936 8555|   201 Miller Street|    Level 15|  North Sydney|       NSW|      2060|Australia|     APAC|
|   Atelier graphique|       40.32.2555|      54, rue Royale|        NULL|  

In [57]:
calendar.count()

252

In [58]:
sales_data.count()

2823

In [59]:
customers.count()

92

In [60]:
master = spark.sql('''

                  SELECT DISTINCT s.ordernumber, c.city
                  FROM sales_data s
                  INNER JOIN customers c ON s.CUSTOMERNAME=c.CUSTOMERNAME

                  ''')
master.show()

+-----------+-------------+
|ordernumber|         city|
+-----------+-------------+
|      10300|    Frankfurt|
|      10385|   San Rafael|
|      10241|   Strasbourg|
|      10182|   San Rafael|
|      10140|   Burlingame|
|      10153|       Madrid|
|      10293|       Torino|
|      10161|      Aaarhus|
|      10406|    Kobenhavn|
|      10414|       Boston|
|      10311|       Madrid|
|      10357|   San Rafael|
|      10195| White Plains|
|      10189|     Pasadena|
|      10422|    Allentown|
|      10111|San Francisco|
|      10204|          NYC|
|      10151|         Oulu|
|      10304|   Versailles|
|      10369|   Brickhaven|
+-----------+-------------+
only showing top 20 rows

